In [3]:
import os
import openai
import yaml

import re


import json
import re

# Load API key from a YAML configuration file
with open("config.yaml") as f:
 config_yaml = yaml.load(f, Loader=yaml.FullLoader)

# Initialize the OpenAI client with the API key
client = openai.OpenAI(api_key=config_yaml['token'])

# Set the model name
MODEL = "gpt-4o-mini"

def llm(prompt, stop=["\n"]):
    # Prepare the dialog for the API request
    dialogs = [{"role": "user", "content": prompt}]

    completion = client.chat.completions.create(
        model=MODEL,
        messages=dialogs
    )
    return completion.choices[0].message.content

In [4]:


def generate_sequence_explanation_prompt(original_action):
    prompt = f"""
<TASK>
Based on your breakdown of the action '{original_action}', provide a concise explanation for each of the steps you identified.
</TASK>

<REQUIREMENT>
- Focus on describing the dynamic movement.
- Highlight the necessary coordination between body parts.
</REQUIREMENT>

<EXCLUSION>
- Do not include any description of facial expressions or emotions.
- Focus solely on the action and movement itself.
</EXCLUSION>

<END_SIGNAL>
- When your explanation is finished, signal completion by saying '<SEQUENCEEND>'.
</END_SIGNAL>
    """
    return prompt


def generate_sequence_explanation_prompt_json(sequence_explanation_prompt):
    prompt = f"""
<TASK>
Based on your breakdown of the action 'sequence_explanation_prompt', evaluate fine motion control for the following body parts:
    <BODY_PARTS>
    - spine
    - Left Arm
    - Right Arm
    - Left Leg
    - Right Leg
    </BODY_PARTS>
    
<sequence_explanation_prompt>
{sequence_explanation_prompt}
</sequence_explanation_prompt>

</TASK>



<EXAMPLES>
EXAMPLE1:
{{"body part": "left arm", "description": "The man's left arm remains stationary at his side."}}
{{"body part": "right arm", "description": "The man's right arm moves in a waving motion."}}
{{"body part": "left leg", "description": "The man's left leg is stationary."}}
{{"body part": "right leg", "description": "The man's right leg is lifted slightly off the ground."}}
{{"body part": "spine", "description": "The man's spine moves in a wave-like motion."}}

EXAMPLE2:
{{"body part": "left arm", "description": "The man's left arm is bent at the elbow and held close to his body."}}
{{"body part": "right arm", "description": "The man's right arm moves in a circular motion."}}
{{"body part": "left leg", "description": "The man's left leg is bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee."}}
{{"body part": "spine", "description": "The man's spine moves in a rhythmic motion."}}

EXAMPLE3:
{{"body part": "left arm", "description": "The man's left arm is raised in a bent position."}}
{{"body part": "right arm", "description": "The man's right arm is raised and bent at the elbow."}}
{{"body part": "left leg", "description": "The man's left leg is lifted and bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee and lifted slightly off the ground."}}
{{"body part": "spine", "description": "The spine is arched slightly forward."}}
</EXAMPLES>

<FORMAT>
Ensure the explanation is in the following JSON-like format for each step and body part:

{{
    "step1": [
        {{
            "body part": "left arm",
            "description": "The man's left arm [specific movement description]."
        }},
        {{
            "body part": "right arm",
            "description": "The man's right arm [specific movement description]."
        }},
        {{
            "body part": "left leg",
            "description": "The man's left leg [specific movement description]."
        }},
        {{
            "body part": "right leg",
            "description": "The man's right leg [specific movement description]."
        }},
        {{
            "body part": "spine",
            "description": "The man's spine [specific movement description]."
        }}
    ],
    "step2": [
        {{
            "body part": "left arm",
            "description": "The man's left arm [specific movement description]."
        }},
        {{
            "body part": "right arm",
            "description": "The man's right arm [specific movement description]."
        }},
        {{
            "body part": "left leg",
            "description": "The man's left leg [specific movement description]."
        }},
        {{
            "body part": "right leg",
            "description": "The man's right leg [specific movement description]."
        }},
        {{
            "body part": "spine",
            "description": "The man's spine [specific movement description]."
        }}
    ],
    ...(continue for each step)
}}

Focus on the movement and positioning of each body part, similar to the provided examples. Be concise and avoid vague terms. Use clear and specific descriptions.
</FORMAT>

<REQUIREMENT>
Focus only on these body parts. DO NOT include any details about facial expressions, eyes, or emotions.
Be concise and AVOID providing any reasoning or explanation—focus only on the action of each body part.
</REQUIREMENT>

<END_SIGNAL>
When you finish the explanation for all steps, say '<SEQUENCEEND>'.
</END_SIGNAL>
    """
    return prompt


def sequence_analyze(action):
    sequence_explanation_prompt = generate_sequence_explanation_prompt(action)
    sequence_explanation = llm(sequence_explanation_prompt, stop=["<SEQUENCEEND>"]).split("<SEQUENCEEND>")[0].strip()
    print(sequence_explanation)

    # Use the updated format to generate JSON-like output for each body part
    sequence_explanation_prompt2 = generate_sequence_explanation_prompt_json(sequence_explanation)
    print(sequence_explanation_prompt2)

    sequence_explanation2 = llm(sequence_explanation_prompt2, stop=["<SEQUENCEEND>"]).split("<SEQUENCEEND>")[0].strip()

    # Formatting the output into the desired JSON-like format for each body part
    output = (
            "Action: " + action + "\n" +
            sequence_explanation2 + "\n" +
            "\n"
    )

    # Write to file
    with open("sequence_explanation.txt", "a") as file:
        file.write(output)

    # Print to console
    print(output)

    # Return results as well
    return sequence_explanation2

txt = sequence_analyze("Perform a signature James Bond pose with a dramatic turn and gunpoint.")
print("done")

1. **Initiate the Turn**: Begin by shifting your weight onto your back foot while pivoting on the ball of that foot. This movement requires coordination between the hips and shoulders to initiate the rotation. As you turn, your front foot should point away in alignment with your direction of movement.

2. **Engage Core Muscles**: Activate your core muscles to stabilize your body during the turn. This engagement allows for smooth transitions and control, preventing loss of balance during the dynamic movement.

3. **Extend the Arms**: As you complete the turn, simultaneously extend your dominant arm forward, mimicking the action of drawing a weapon. This extension should be fluid, with the shoulder, elbow, and wrist moving in unison to create a strong, assertive line.

4. **Assume a Stance**: At the apex of your motion, widen your stance slightly to establish a firm base. The legs should be slightly bent, with weight evenly distributed between both feet, enabling quick reactions to any i

In [8]:
def generate_fine_motion_control_prompt(original_action, sequence_explanation):
    prompt = f"""
<TASK>
Generate fine motion control descriptions for each body part based on the action '{original_action}' and the {sequence_explanation}.

</TASK>

<REQUIREMENT>
Focus on describing the specific actions for each body part without providing explanations or reasoning. Avoid vague terms such as "at an appropriate angle" or "adjust for balance." Instead, use precise descriptions like "Raise the right arm to shoulder height." 
Do not include any details about facial expressions, eyes, or emotions.
I don't like overly abstract language, such as "as if aiming a firearm." Understanding this action requires a high level of comprehension that general models may not possess, as this description does not clearly specify what kind of action it is. Please use more specific action descriptions.
</REQUIREMENT>

<EXAMPLES>
EXAMPLE1:
{{"body part": "left arm", "description": "The man's left arm remains stationary at his side."}}
{{"body part": "right arm", "description": "The man's right arm moves in a waving motion."}}
{{"body part": "left leg", "description": "The man's left leg is stationary."}}
{{"body part": "right leg", "description": "The man's right leg is lifted slightly off the ground."}}
{{"body part": "spine", "description": "The man's spine moves in a wave-like motion."}}

EXAMPLE2:
{{"body part": "left arm", "description": "The man's left arm is bent at the elbow and held close to his body."}}
{{"body part": "right arm", "description": "The man's right arm moves in a circular motion."}}
{{"body part": "left leg", "description": "The man's left leg is bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee."}}
{{"body part": "spine", "description": "The man's spine moves in a rhythmic motion."}}

EXAMPLE3:
{{"body part": "left arm", "description": "The man's left arm is raised in a bent position."}}
{{"body part": "right arm", "description": "The man's right arm is raised and bent at the elbow."}}
{{"body part": "left leg", "description": "The man's left leg is lifted and bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee and lifted slightly off the ground."}}
{{"body part": "spine", "description": "The spine is arched slightly forward."}}
</EXAMPLES>

<INPUT>
Using the action original_action and the sequence_explanation, generate fine motion control descriptions for the following body parts:

- spine
- left arm
- right arm
- left leg
- right leg

<original_action>
{original_action}
</original_action>

<sequence_explanation>
{sequence_explanation}
</sequence_explanation>

<FORMAT>
For each body part, provide a concise and specific action description in the following JSON format:

{{
    "body part": "left arm",
    "description": "The man's left arm [specific movement description]."
}},
{{
    "body part": "right arm",
    "description": "The man's right arm [specific movement description]."
}},
{{
    "body part": "left leg",
    "description": "The man's left leg [specific movement description]."
}},
{{
    "body part": "right leg",
    "description": "The man's right leg [specific movement description]."
}},
{{
    "body part": "spine",
    "description": "The man's spine [specific movement description]."
}}
</FORMAT>
</INPUT>

<END_SIGNAL>
When you finish, say '<CONTROLEND>'.
</END_SIGNAL>
"""
    return prompt


def analyze_fine_moton_control_txt(action):
    # Step 1: Get sequence explanation
    sequence_explanation_prompt = generate_sequence_explanation_prompt(action)
    sequence_explanation = llm(sequence_explanation_prompt, stop=["<SEQUENCEEND>"]).split("<SEQUENCEEND>")[0].strip()

    # Step 2: Evaluate fine motion control
    fine_moton_control_prompt = generate_fine_motion_control_prompt(action, sequence_explanation)
    control_evaluation = llm(fine_moton_control_prompt, stop=["<CONTROLEND>"]).split("<CONTROLEND>")[0].strip()

    # Parse the JSON objects from the control evaluation
    json_objects = re.findall(r'\{[^}]+\}', control_evaluation)
    control_results = [json.loads(obj) for obj in json_objects]

    # Output to file as well as print
    # output = {
    #     "Action": action,
    #     "Sequence Explanation": sequence_explanation,
    #     "Fine Motion Control Evaluation": control_results
    # }
    output = control_results

    # Write to file
    with open("analyze_fine_moton_control.json", "a") as file:
        json.dump(output, file, ensure_ascii=False, indent=2)
        file.write("\n")

    # Print to console
    print(json.dumps(output, ensure_ascii=False, indent=2))

    # Return results as well
    return sequence_explanation, control_results

actions_to_test = [
    "Perform a signature James Bond pose with a dramatic turn and gunpoint.",
    # "Fight with the fluid precision and power of Bruce Lee.",
]

# Test each action and save the results in the JSON file
for action in actions_to_test:
    sequence_explanation, control_results = analyze_fine_moton_control_txt(action)

 

[
  {
    "body part": "left arm",
    "description": "The man's left arm is bent at the elbow and positioned close to his body for stabilization."
  },
  {
    "body part": "right arm",
    "description": "The man's right arm is extended outward at shoulder height, with the wrist straight and fingers positioned as if gripping a firearm."
  },
  {
    "body part": "left leg",
    "description": "The man's left leg is firmly planted on the ground, providing balance during the pose."
  },
  {
    "body part": "right leg",
    "description": "The man's right leg is slightly bent at the knee, maintaining a shoulder-width stance for stability."
  },
  {
    "body part": "spine",
    "description": "The man's spine is slightly twisted to follow the direction of the turn, enhancing the dramatic effect of the pose."
  }
]


In [7]:
def check_trajectory_control_with_llm(action):
    # First call: Let the LLM think using chain of thought reasoning
    first_prompt = f"""
    Examine the action: "{action}".

    Trajectory control is only necessary if the action includes explicit instructions for movement along a specific two-dimensional path. This typically includes instructions like "turn", "move in a circle", "zigzag", or "change direction", which clearly involve a shift in direction or orientation.

    If an action involves simple movement in a straight line (such as "run forward" or "jump in place") or basic rotations (like "spin in place"), trajectory control is not required because there is no complex or changing path to follow.

    Consider whether the action explicitly requires a change in direction or follows a curved or angular path, then conclude if trajectory control is necessary. If the path is simple or linear, trajectory control is not needed.

    Explain your reasoning step by step and conclude with whether trajectory control is required.
    """

    first_response = llm(first_prompt)

    # Second call: Ask LLM to give a simple boolean response based on its reasoning
    second_prompt = f"""
Based on your previous analysis: "{first_response}", answer with only 'True' if trajectory control is required, and 'False' if it is not.
"""
    second_response = llm(second_prompt).strip()

    return second_response


# Convert the second response to a boolean value
def response_to_bool(response):
    if response.lower() == "true":
        return True
    elif response.lower() == "false":
        return False
    else:
        raise ValueError("Unexpected response from LLM: " + response)

#
# # List of action cases to test
# actions = [
#     "Run forward, then jump in place",                       # Example 1
#     "Walk forward and make a left turn",                     # Example 2
#     "Stand still and wave your hand",                        # Example 3
#     "Run up, then kneel and slide backward",                 # Example 4
#     "Walk around in a circle",                               # Example 5
#     "Spin in place on one foot",                             # Example 6
#     "Crawl under the table",                                 # Example 7
#     "Run forward and stop at the door",                      # Example 8
#     "Jump forward and then turn around",                     # Example 9
#     "Dance freely with no set path",                         # Example 10
#     "Climb up the ladder",                                   # Example 11
#     "Roll on the ground twice",                              # Example 12
#     "Run forward, kneel, and slide in a straight line",      # Example 13
#     "Hop in a zigzag pattern",                               # Example 14
#     "Jump sideways then spin 180 degrees",                   # Example 15
#     "Walk straight then stop and jump on one leg",           # Example 16
#     "Swim forward in a straight line",                       # Example 17
#     "Skate in a figure-eight pattern",                       # Example 18
#     "Step backward three times then kneel",                  # Example 19
#     "Jump forward and spin in a circle mid-air"              # Example 20
# ]
#
# # Test trajectory control for each action using LLM
# for action in actions:
#     print(f"Action: {action}")
#     reasoning = check_trajectory_control_with_llm(action)
#     result_bool = response_to_bool(reasoning)
#     print(f"Needs Trajectory Control: {result_bool}")
#     print("=======================================")
